In [8]:
from qdrant_client import QdrantClient, models
import numpy as np

In [2]:
from fastembed import TextEmbedding

In [58]:
client = QdrantClient("http://localhost:6333")

In [4]:
embedder = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\dehzhang\Desktop\GitHub-Repos\llm-zoomcamp-2025\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dehzhang\AppData\Local\Temp\fastembed_cache\models--xenova--jina-embeddings-v2-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to acti

In [18]:
query = "I just discovered the course. Can I join now?"

In [29]:
embedding = next(embedder.embed([query]))

In [30]:
# Q1
round(np.min(embedding), 2)

np.float64(-0.12)

In [20]:
doc = 'Can I still join the course after the start date?'

In [31]:
doc_embedding = next(embedder.embed([doc]))

In [34]:
#Q2
embedding.dot(doc_embedding) / (np.linalg.norm(embedding) * np.linalg.norm(doc_embedding))

np.float64(0.9008528895674548)

In [35]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [38]:
texts = [d["text"] for d in documents]
V = np.array(list(embedder.embed(texts)), dtype=np.float32)

In [ ]:
#Q3
best_idx = np.argmax(V.dot(embedding))
best_idx

np.int64(1)

In [ ]:
#Q4
full_texts = [d["question"] + " " + d["text"] for d in documents]
V_full = np.array(list(embedder.embed(full_texts)), dtype=np.float32)
best_idx_full = np.argmax(V_full.dot(embedding))
best_idx_full

np.int64(0)

In [ ]:
#Q5
min([m["dim"] for m in TextEmbedding.list_supported_models() if "dim" in m])

384

In [53]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [57]:
points = []
id = 0

for doc in documents:

    text = doc['question'] + ' ' + doc['text']

    point = models.PointStruct(
        id=id,
        vector=models.Document(text=text, model="BAAI/bge-small-en"), 
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "course": course['course']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1

In [59]:
collection_name = "llm-zoomcamp-2025"

# Create the collection with specified vector parameters
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=384,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

C:\Users\dehzhang\AppData\Local\Temp\ipykernel_16440\2433125970.py:4: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [60]:
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]c:\Users\dehzhang\Desktop\GitHub-Repos\llm-zoomcamp-2025\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dehzhang\AppData\Local\Temp\fastembed_cache\models--Qdrant--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Fetching 5 files

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [61]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model="BAAI/bge-small-en" 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [68]:
#Q6
search(query)

QueryResponse(points=[ScoredPoint(id=14, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'course': 'mlops-zoomcamp'}, vector=None, shard_key=None, order_value=None)])